In [1]:
import orca
import urbansim

import datasources
import variables

Model template is: zone. Operating on zones with zone_id.


In [2]:
j = orca.get_table('jobs').to_frame()
z = orca.get_table('zones').to_frame()

In [3]:
z.columns

Index([u'avg_nonres_m2_rent', u'avg_residential_rent',
       u'avg_residential_value', u'avg_year_built', u'land_km2', u'm2_per_job',
       u'non_residential_m2', u'proportion_undevelopable',
       u'residential_units', u'total_household_population',
       u'total_number_of_households', u'total_number_of_jobs',
       u'travel_time_to_cbd_drive', u'x', u'y', u'households', u'jobs',
       u'du_spaces', u'vacant_du_spaces', u'z_id', u'vacant_job_spaces',
       u'all_zones', u'vacant_residential_units', u'job_spaces',
       u'ln_residential_units'],
      dtype='object')

In [4]:
expvars = ['total_household_population', 'travel_time_to_cbd_drive', 'avg_year_built', 'total_number_of_jobs']
dcm = urbansim.models.dcm.MNLDiscreteChoiceModel(model_expression=expvars,
                                                 sample_size=50,
                                                 probability_mode='single_chooser',
                                                 choice_mode='aggregate',
                                                 estimation_sample_size=1000)

In [5]:
dcm.fit(j, z, 'zone_id')

/home/janowicz/udst/urbansim/urbansim/urbanchoice/interaction.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  alts_sample['join_index'] = np.repeat(choosers.index.values, SAMPLE_SIZE)


{'convergence': -3339.915740752933,
 'null': -3912.023005428176,
 'ratio': 0.14624332829367537}

In [8]:
j = j[j.sector_id==1]

In [7]:
from patsy import dmatrix
from urbansim.urbanchoice import interaction, mnl
from urbansim.models import util

In [24]:
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target

In [21]:
_, merged, chosen = interaction.mnl_interaction_dataset(
            j, z, 50, j['zone_id'])

/home/janowicz/udst/urbansim/urbansim/urbanchoice/interaction.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  alts_sample['join_index'] = np.repeat(choosers.index.values, SAMPLE_SIZE)


In [23]:
merged.columns

Index([u'avg_nonres_m2_rent', u'avg_residential_rent',
       u'avg_residential_value', u'avg_year_built', u'land_km2', u'm2_per_job',
       u'non_residential_m2', u'proportion_undevelopable',
       u'residential_units', u'total_household_population',
       u'total_number_of_households', u'total_number_of_jobs',
       u'travel_time_to_cbd_drive', u'x', u'y', u'households', u'jobs',
       u'du_spaces', u'vacant_du_spaces', u'z_id', u'vacant_job_spaces',
       u'all_zones', u'vacant_residential_units', u'job_spaces',
       u'ln_residential_units', u'join_index', u'sector_id', u'zone_id',
       u'y_r', u'x_r', u'all_jobs'],
      dtype='object')

In [25]:
chosen.shape

(7676, 50)

In [9]:
##scikit-urbansim is only useful for leveraging external scikit-learn community libraries
## we still want to be able to go back to the main urbansim interface for simulation.  
## just not the lcm_simulate in utils.  Execute simulation on the model object itself from within orca/models.py
## model-object based approach facilitates ensembling, and grouping models together.
## zone model ensemble example now!
## the scikit-learn version of models all need to assume that the interaction dataset is X
##     (so calculate the interaction dataset up front)
## separate the probability calculation from the simulation.  Simulate function is generic and separate

def create_lcm_training_data(model_expression, choosers, alternatives, alts_sample_size, current_choice):
    
    current_choice = choosers[current_choice]
    
    _, merged, chosen = interaction.mnl_interaction_dataset(
                choosers, alternatives, alts_sample_size, current_choice)

    str_model_expression = util.str_model_expression(model_expression, add_constant=False)
    model_design = dmatrix(str_model_expression, data=merged, return_type='dataframe')

    X = model_design.as_matrix()
    y = chosen.ravel()
    
    return X, y

In [10]:
X, y = create_lcm_training_data(expvars, j, z, 50, 'zone_id')

In [38]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression().fit(X,y)

In [44]:
import numpy as np
import pandas as pd
from urbansim.models import MNLDiscreteChoiceModel
from urbansim.utils import misc


from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.multiclass import unique_labels
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.estimator_checks import check_estimator

class MnlClassifier(MNLDiscreteChoiceModel, BaseEstimator, ClassifierMixin):

    def __init__(self, model_expression='', sample_size=50):
         super(MnlClassifier, self).__init__(model_expression, sample_size)
        
    def fit(self, X, y):
        ##  X here should be MNLDiscreteChoiceModel.fit, with pre-merged choosers/alts...
        
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)

        self.X_ = X
        self.y_ = y
        
        num_agents = len(X) / self.sample_size
        chosen = y.reshape((num_agents, self.sample_size))
        
        self.log_likelihoods, self.fit_parameters = mnl.mnl_estimate(
            X, chosen, self.sample_size)
        
        # Return the classifier
        return self
    
    def probabilities(self, X):
        probabilities = mnl.mnl_simulate(
            X,
            list(lcm.fit_parameters['Coefficient']),
            numalts=len(X), returnprobs=True)
        return probabilities.flatten()
    
    def predict_proba(self):
        pass
    
    def predict(self, X):
        ##  Actually, this should be a separate simulation thing (outside of the object)
        # Check is fit had been called
        check_is_fitted(self, ['X_', 'y_'])

        # Input validation
        X = check_array(X)
        
        probs = self.probabilities(X)
        
        return np.random.choice(
                    np.arange(len(probs)), p=probs / probs.sum())
    
#check_estimator(MnlClassifier)

In [45]:
lcm = MnlClassifier().fit(X, y)

In [35]:
lcm.fit_parameters['Coefficient'][1]

0.02354313100529035

In [47]:
lcm.probabilities(X)

array([  8.99729706e-07,   5.76687411e-06,   1.18368201e-06, ...,
         1.25697494e-06,   1.90030532e-06,   1.46803129e-06])

In [48]:
lcm.predict(X)

330032

In [29]:
list(lcm.fit_parameters['Coefficient'])

[6.169016069863082e-06,
 0.02354313100529035,
 0.0100955754795859,
 4.5110566896538025e-06]

In [2]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

from datashader import transfer_functions as tf
from datashader.colors import Greys9
Greys9_r = list(reversed(Greys9))[:-2]

from bokeh.tile_providers import STAMEN_TERRAIN

import datashader as ds
from datashader.bokeh_ext import InteractiveImage
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from IPython.core.display import HTML, display

def plot_agents(df, name, sum_attrib=None, theme=Hot):
    df['ones'] = 1
    region = x_range, y_range = ((df.x.min(),df.x.max()), (df.y.min(),df.y.max()))

    plot_width  = int(750)
    plot_height = int(plot_width//1.2)

    def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
        p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
            x_range=x_range, y_range=y_range, outline_line_color=None,
            min_border=0, min_border_left=0, min_border_right=0,
            min_border_top=0, min_border_bottom=0, **plot_args)

        p.axis.visible = False
        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
        return p

    options = dict(line_color=None, fill_color='blue', size=5)

    background = "black"
    export = partial(export_image, export_path="export", background=background)
    cm = partial(colormap_select, reverse=(background=="black"))

    def create_image(x_range, y_range, w=plot_width, h=plot_height):
        cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
        if sum_attrib:
            agg = cvs.points(df, 'x', 'y',  ds.count(sum_attrib))
        else:
            agg = cvs.points(df, 'x', 'y',  ds.count('ones'))
        img = tf.shade(agg, cmap=theme, how='eq_hist')
        return tf.dynspread(img, threshold=0.5, max_px=4)

    p = base_plot(background_fill_color=background)
    #p.add_tile(STAMEN_TERRAIN)
    export(create_image(*region), name)
    return InteractiveImage(p, create_image)

def plot_agents_by_cat(df, name):
    df['ones'] = 1
    region = x_range, y_range = ((df.x.min(),df.x.max()), (df.y.min(),df.y.max()))

    plot_width  = int(750)
    plot_height = int(plot_width//1.2)

    def base_plot(tools='pan,wheel_zoom,reset',plot_width=plot_width, plot_height=plot_height, **plot_args):
        p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
            x_range=x_range, y_range=y_range, outline_line_color=None,
            min_border=0, min_border_left=0, min_border_right=0,
            min_border_top=0, min_border_bottom=0, **plot_args)

        p.axis.visible = False
        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
        return p

    options = dict(line_color=None, fill_color='blue', size=5)

    background = "black"
    export = partial(export_image, export_path="export", background=background)
    cm = partial(colormap_select, reverse=(background=="black"))

    def create_image(x_range, y_range, w=plot_width, h=plot_height):
        cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
        color_key = color_key = {'low':'aqua', 'high':'red'}

        agg = cvs.points(df, 'x', 'y',  ds.count_cat('proba_class'))

        img = tf.shade(agg, color_key=color_key, how='eq_hist')
        return tf.dynspread(img, threshold=0.5, max_px=4)

    p = base_plot(background_fill_color=background)
    #p.add_tile(STAMEN_TERRAIN)
    export(create_image(*region), name)
    return InteractiveImage(p, create_image)

Loading BokehJS ...